In [6]:
import sys
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Tool")
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Strategy")
from cn_general_tool import *
from cn_stock_strategy_factor import *

In [7]:
sd, ed = get_sd_ed('2019-01-01', '2024-11-25')

In [15]:
sql = """
SELECT 
    date, 
    instrument, 
    volume, 
    m_rank(volume, 20, method:='min', ascending:=true) AS _n_rank, 
    (_n_rank * 2 - 20) / 20 AS norm_v, 
FROM cn_stock_index_bar1d
WHERE instrument =  '000852.SH'
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date
"""

In [16]:
df = get_dai_df(sql, sd, ed)
df

,date,instrument,volume,norm_v
0,2019-01-02,000852.SH,66272240,-0.8
1,2019-01-03,000852.SH,80494980,0.1
2,2019-01-04,000852.SH,103470812,0.9
3,2019-01-07,000852.SH,114553618,1.0
4,2019-01-08,000852.SH,107411023,0.9
...,...,...,...,...
1425,2024-11-19,000852.SH,273167526,-0.8
1426,2024-11-20,000852.SH,299975821,-0.4
1427,2024-11-21,000852.SH,299655111,-0.5
1428,2024-11-22,000852.SH,342129117,0.1


In [17]:
def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.holding_days = 5
    context.value = 0.8

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    if context.trading_day_index % context.holding_days != 0:
        return

    current_date = data.current_dt.strftime("%Y-%m-%d")
    current_day_data = context.data[context.data["date"] == current_date]
    
    target_hold_instruments = list(current_day_data["instrument"])[0]
    current_hold_instruments = set(context.get_account_positions().keys())
    
    rank = current_day_data['norm_v'].iloc[0]

    if rank < context.value and target_hold_instruments in current_hold_instruments:
        context.order_target_percent(target_hold_instruments, 0)

    if rank >= context.value and target_hold_instruments not in current_hold_instruments:
        context.order_target_percent(target_hold_instruments, 1)


def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000,
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""中证1000指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2024-12-02 15:35:20] [info     ] bigtrader.v34 开始运行 ..


[2024-12-02 15:35:20] [info     ] 2019-01-02, 2024-11-25, , , instruments=1
[2024-12-02 15:35:21] [info     ] bigtrader module V2.2.0
[2024-12-02 15:35:21] [info     ] bigtrader engine v1.10.10 2024-11-26
[2024-12-02 15:35:26] [info     ] backtest done, raw_perf_ds:dai.DataSource("_1e795ef3d01e46f19eeb06ce351d1415")


[2024-12-02 15:35:29] [info     ] bigtrader.v34 运行完成 [8.660s].
